In [1]:
import numpy as np
import pandas as pd
%cd

/Users/harshit


In [2]:
!pwd

/Users/harshit


In [3]:
%cd Downloads/ML_Projects/Sentiment Analysis

/Users/harshit/Downloads/ML_Projects/Sentiment Analysis


In [4]:
%cd sentiment-analysis-on-movie-reviews

/Users/harshit/Downloads/ML_Projects/Sentiment Analysis/sentiment-analysis-on-movie-reviews


In [5]:
!pwd

/Users/harshit/Downloads/ML_Projects/Sentiment Analysis/sentiment-analysis-on-movie-reviews


In [6]:
%ls

Movie Review Prediction.csv  test.tsv*
sampleSubmission.csv*        train.tsv*


In [7]:
data = pd.read_table('train.tsv')

In [8]:
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [9]:
data.shape

(156060, 4)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156060 entries, 0 to 156059
Data columns (total 4 columns):
PhraseId      156060 non-null int64
SentenceId    156060 non-null int64
Phrase        156060 non-null object
Sentiment     156060 non-null int64
dtypes: int64(3), object(1)
memory usage: 4.8+ MB


In [11]:
import string

In [12]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))

def preprocessor(s):
    s = s.split()
    for i in range(0,len(s)):
        k = s.pop(0)
        if k not in string.punctuation and k not in stopwords:
                s.append(lemmatizer.lemmatize(k).lower())    
    return s

In [13]:
X = data['Phrase']
y = data['Sentiment']

In [14]:
print(preprocessor(X[81]))

['even', 'fan', 'ismail', 'merchant', "'s", 'work', 'i', 'suspect', 'would', 'hard', 'time', 'sitting', 'one']


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(analyzer = preprocessor)
X_train_count = count_vect.fit_transform(X_train)
X_train_count.shape

(109242, 15385)

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer
tfid_transformer = TfidfTransformer()
X_train_tfid = tfid_transformer.fit_transform(X_train_count)
X_train_tfid.shape

(109242, 15385)

In [18]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier().fit(X_train_tfid, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [19]:
X_test = count_vect.transform(X_test)
X_test_tfidf = tfid_transformer.transform(X_test)
predictions = clf.predict(X_test_tfidf)

In [20]:
from sklearn.metrics import classification_report
print(classification_report(predictions, y_test))

              precision    recall  f1-score   support

           0       0.38      0.44      0.41      1786
           1       0.48      0.53      0.50      7352
           2       0.79      0.71      0.75     26628
           3       0.49      0.54      0.52      8906
           4       0.38      0.48      0.43      2146

   micro avg       0.63      0.63      0.63     46818
   macro avg       0.50      0.54      0.52     46818
weighted avg       0.65      0.63      0.64     46818



In [21]:
test_data = pd.read_table('test.tsv')

In [22]:
test_data.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [23]:
phrase_id = test_data['PhraseId'].values
test_data = count_vect.transform(test_data['Phrase'])
test_data_tfidf = tfid_transformer.transform(test_data)
test_predictions = clf.predict(test_data_tfidf)

In [24]:
test_predictions.shape

(66292,)

In [25]:
prediction = pd.DataFrame({'PhraseId':phrase_id, 'Sentiment' : test_predictions})

In [26]:
prediction.head()

,PhraseId,Sentiment
0,156061,1
1,156062,1
2,156063,3
3,156064,3
4,156065,3


In [27]:
csv_file = 'Movie Review Prediction.csv'
prediction.to_csv(csv_file, index = False)
print('Saved File : ' + csv_file)

Saved File : Movie Review Prediction.csv
